# Regular Genomics Project **RNA Localisation**


---

**Problem definition**:

TODO


### **1.** First steps with data

Firstly, we import several necessary packages and load in our data:

In [1]:
import pandas as pd
import numpy as np
from keras.utils import to_categorical
import itertools
from collections import OrderedDict
import os
import sys
from keras.utils import pad_sequences
from sklearn.model_selection import KFold, StratifiedKFold
from GeneWrapper import Gene_Wrapper

2023-06-13 12:12:04.543948: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-13 12:12:04.574705: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-13 12:12:04.575716: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-13 12:12:05.277794: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Defining a few parameters
max_len = 4000
nb_filters = 32
filters_length = 10
pooling_size = 3
lstm_units = 32
lower_bound = 0
upper_bound = 4000
nb_classes = 9 # because we have 9 localisations

In [3]:
# Initializing test set
np.random.seed(3)
data_org = pd.read_csv('~/Downloads/final_data.csv')
test_data = data_org.sample(frac=0.1)
train_data = data_org.drop(test_data.index) # TODO: note: we also have to preprocess the test set similary
# TODO: colab

data_org

,ERM,KDEL,LMA,MITO,NES,NIK,NLS,NUCP,OMM,gene_id,gene_biotype,seq,struct,m6A_5UTR,m6A_CDS,m6A_3UTR
0,57.045409,35.456782,22.008215,12.355106,22.789983,24.241731,16.970436,29.348389,54.916891,ENSG00000000003,protein_coding,ACCTTGTATTAGGTATTTATTTCCACAAAAGTTTGATGCTTACAAC...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0,1
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.914814,0.244517,ENSG00000000005,protein_coding,TGTGCACAGAAGTTATATACATATATGGGTATATCTATGTAACAAA...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0,0
2,17.449430,34.151539,29.338431,22.237585,24.682767,43.612551,38.683963,35.678476,25.348560,ENSG00000000419,protein_coding,TACTTTATGCAAAAAAAAATATACATTTATTTATAGGTCTCAATAC...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0,0
3,3.830180,2.576734,5.737850,0.761343,2.786808,2.784356,3.382682,2.463676,2.819269,ENSG00000000457,protein_coding,TGACTTTCAAACCATTTTAATATTTCAAATATTCCAGAACAATCCC...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,9,5
4,13.132915,8.782925,10.061390,3.012459,8.821250,6.721117,10.827253,8.005113,6.849962,ENSG00000000460,protein_coding,AACCCGCTCGGGTCCCCTTCCACACTGTGGAAGCTTTGTTCTTTCG...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13805,0.000000,0.000000,0.506809,0.075893,0.050764,0.000000,0.056586,0.000000,0.000000,ENSG00000281883,protein_coding,GGGAAGAAAGGAGCCTGACTCTTATGATGGAATAACCACAAATCAG...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,2,1
13806,0.105452,0.087130,0.171187,0.016101,0.079057,0.669947,0.171672,0.000000,0.254546,ENSG00000282034,protein_coding,GTGTCGGACGGCATGACAGGCAGCAATCCTGTGTCCCCTGCCTCAT...,"[0.37599998712539673, 0.0, 0.07500000298023224...",0,66,4
13807,0.037093,0.444844,0.425368,0.254467,0.323794,1.803249,1.435483,0.249590,0.483645,ENSG00000282827,protein_coding,ATGGCGGGGACCTCCGCGCCAGGCAGCAAGAGGCGGAGCGAGCCCC...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0,0
13808,0.000000,0.519421,2.119115,0.360450,0.887939,0.345021,1.274465,0.741954,0.460649,ENSG00000282936,protein_coding,CTACTGGAACGCCCCCCTCAATCTAGCCTCCCCCACATAACTCTCT...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0,2


In [4]:
sum_vec = train_data.iloc[:, :9].sum(axis=1)
data2 = train_data.iloc[:, :9].divide(sum_vec, axis='index')
train_data_no_struct = pd.concat([data2, train_data['seq']], axis=1)
train_data_no_struct




,ERM,KDEL,LMA,MITO,NES,NIK,NLS,NUCP,OMM,seq
0,0.207338,0.128871,0.079991,0.044906,0.082833,0.088109,0.061681,0.106670,0.199601,ACCTTGTATTAGGTATTTATTTCCACAAAAGTTTGATGCTTACAAC...
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.922605,0.077395,TGTGCACAGAAGTTATATACATATATGGGTATATCTATGTAACAAA...
2,0.064346,0.125935,0.108187,0.082002,0.091019,0.160823,0.142649,0.131566,0.093474,TACTTTATGCAAAAAAAAATATACATTTATTTATAGGTCTCAATAC...
3,0.141112,0.094932,0.211394,0.028049,0.102672,0.102581,0.124625,0.090767,0.103868,TGACTTTCAAACCATTTTAATATTTCAAATATTCCAGAACAATCCC...
4,0.172315,0.115240,0.132014,0.039526,0.115743,0.088187,0.142063,0.105034,0.089878,AACCCGCTCGGGTCCCCTTCCACACTGTGGAAGCTTTGTTCTTTCG...
...,...,...,...,...,...,...,...,...,...,...
13805,0.000000,0.000000,0.734451,0.109981,0.073566,0.000000,0.082002,0.000000,0.000000,GGGAAGAAAGGAGCCTGACTCTTATGATGGAATAACCACAAATCAG...
13806,0.067811,0.056029,0.110082,0.010354,0.050838,0.430809,0.110393,0.000000,0.163685,GTGTCGGACGGCATGACAGGCAGCAATCCTGTGTCCCCTGCCTCAT...
13807,0.006797,0.081510,0.077941,0.046627,0.059330,0.330415,0.263028,0.045733,0.088620,ATGGCGGGGACCTCCGCGCCAGGCAGCAAGAGGCGGAGCGAGCCCC...
13808,0.000000,0.077421,0.315861,0.053726,0.132350,0.051426,0.189963,0.110591,0.068661,CTACTGGAACGCCCCCCTCAATCTAGCCTCCCCCACATAACTCTCT...


In [5]:
# One hot encode the 'seq' attribute of the above table
mapping = { 
    'A': 0,
    'C': 1,
    'G': 2,
    'T': 3
}

mapping_localisations = {
    'ERM':  0,
    'KDEL': 1,
    'LMA':  2,
    'MITO': 3,
    'NES':  4,
    'NIK':  5,
    'NLS':  6,
    'NUCP': 7,
    'OMM':  8
}

one_hot_encode_lam = lambda seq: to_categorical([mapping[x] for x in seq])
data_one = train_data_no_struct['seq'].apply(one_hot_encode_lam)

data_one

# Now just injecting this modified 'seq' back into the pandas frame 
data_one_no_struct =  pd.concat([train_data_no_struct.iloc[:, :9], data_one], axis=1)

data_one_no_struct

,ERM,KDEL,LMA,MITO,NES,NIK,NLS,NUCP,OMM,seq
0,0.207338,0.128871,0.079991,0.044906,0.082833,0.088109,0.061681,0.106670,0.199601,"[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [..."
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.922605,0.077395,"[[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0], [..."
2,0.064346,0.125935,0.108187,0.082002,0.091019,0.160823,0.142649,0.131566,0.093474,"[[0.0, 0.0, 0.0, 1.0], [1.0, 0.0, 0.0, 0.0], [..."
3,0.141112,0.094932,0.211394,0.028049,0.102672,0.102581,0.124625,0.090767,0.103868,"[[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0], [..."
4,0.172315,0.115240,0.132014,0.039526,0.115743,0.088187,0.142063,0.105034,0.089878,"[[1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [..."
...,...,...,...,...,...,...,...,...,...,...
13805,0.000000,0.000000,0.734451,0.109981,0.073566,0.000000,0.082002,0.000000,0.000000,"[[0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 0.0], [..."
13806,0.067811,0.056029,0.110082,0.010354,0.050838,0.430809,0.110393,0.000000,0.163685,"[[0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0], [..."
13807,0.006797,0.081510,0.077941,0.046627,0.059330,0.330415,0.263028,0.045733,0.088620,"[[1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0], [..."
13808,0.000000,0.077421,0.315861,0.053726,0.132350,0.051426,0.189963,0.110591,0.068661,"[[0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0], [..."


In [31]:
#print(mapping_localisations.keys())
#print(list(mapping_localisations.keys()))
#print(data_one_no_struct[mapping_localisations.keys()])
#print(data_one_no_struct[mapping_localisations.keys()].values)

dict_keys(['ERM', 'KDEL', 'LMA', 'MITO', 'NES', 'NIK', 'NLS', 'NUCP', 'OMM'])
['ERM', 'KDEL', 'LMA', 'MITO', 'NES', 'NIK', 'NLS', 'NUCP', 'OMM']
            ERM      KDEL       LMA      MITO       NES       NIK       NLS  \
0      0.207338  0.128871  0.079991  0.044906  0.082833  0.088109  0.061681   
1      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2      0.064346  0.125935  0.108187  0.082002  0.091019  0.160823  0.142649   
3      0.141112  0.094932  0.211394  0.028049  0.102672  0.102581  0.124625   
4      0.172315  0.115240  0.132014  0.039526  0.115743  0.088187  0.142063   
...         ...       ...       ...       ...       ...       ...       ...   
13805  0.000000  0.000000  0.734451  0.109981  0.073566  0.000000  0.082002   
13806  0.067811  0.056029  0.110082  0.010354  0.050838  0.430809  0.110393   
13807  0.006797  0.081510  0.077941  0.046627  0.059330  0.330415  0.263028   
13808  0.000000  0.077421  0.315861  0.053726  0.132350  0.051426

In [6]:

# Additional ordinal encoding of the 'seq' attribute 

gene_data = train_data['seq']

def label_dist(dist):
    # TODO: what is this
    assert (len(dist) == 4)
    return np.array(dist) / np.sum(dist)

encoding_seq = OrderedDict([
    ('UNK', [0, 0, 0, 0]),
    ('A', [1, 0, 0, 0]),
    ('C', [0, 1, 0, 0]),
    ('G', [0, 0, 1, 0]),
    ('T', [0, 0, 0, 1]),
    ('N', [0.25, 0.25, 0.25, 0.25]),  # A or C or G or T
])

encoding_keys = list(encoding_seq.keys())
seq_encoding_vectors = np.array(list(encoding_seq.values()))
encoding_vectors = seq_encoding_vectors

X = pad_sequences([[encoding_keys.index(c) for c in gene] for gene in gene_data],
                    maxlen=max_len,
                    dtype=np.int8, value=encoding_keys.index('UNK'))  # , truncating='post')

y = data_one_no_struct[mapping_localisations.keys()].values

# y = np.array([label_dist(gene.dist) for gene in gene_data])

#template = [0] * 24  # TODO: why 24? # dim([a,c,g,t]) * dim([f,t,i,h,m,s])
#combined_encoding = OrderedDict()
#combined_encoding['UNK'] = template
#for i, (key_seq, key_ann) in enumerate(
#        itertools.product(['A', 'C', 'G', 'T', 'N'], ['F', 'T', 'I', 'H', 'M', 'S'])):
#    tmp = template.copy()
#    if key_seq == 'N':
#        for n in ['A', 'C', 'G', 'T']:
#            tmp[np.nonzero(combined_encoding[n + key_ann])[0][0]] = 0.25
#        combined_encoding[key_seq + key_ann] = tmp
#    else:
#        tmp[i] = 1  # normal one-hot encoding as it is...
#        combined_encoding[key_seq + key_ann] = tmp
#encoding_keys = list(combined_encoding.keys())
#encoding_vectors = np.array(list(combined_encoding.values()))
#encoding_vectors
##print(len(encoding_vectors))
#encoding_keys

In [17]:
# TODO: what is this?
# check

# Generating one-hot encoding sequences
 


"""
encoding_seq = OrderedDict([
    ('UNK', [0, 0, 0, 0]),
    ('A', [1, 0, 0, 0]),
    ('C', [0, 1, 0, 0]),
    ('G', [0, 0, 1, 0]),
    ('T', [0, 0, 0, 1]),
    ('N', [0.25, 0.25, 0.25, 0.25]),  # A or C or G or T
])

template = [0] * 24  # TODO: why 24? # dim([a,c,g,t]) * dim([f,t,i,h,m,s])
combined_encoding = OrderedDict()
combined_encoding['UNK'] = template
for i, (key_seq, key_ann) in enumerate(
        itertools.product(['A', 'C', 'G', 'T', 'N'], ['F', 'T', 'I', 'H', 'M', 'S'])):
    tmp = template.copy()
    if key_seq == 'N':
        for n in ['A', 'C', 'G', 'T']:
            tmp[np.nonzero(combined_encoding[n + key_ann])[0][0]] = 0.25
        combined_encoding[key_seq + key_ann] = tmp
    else:
        tmp[i] = 1  # normal one-hot encoding as it is...
        combined_encoding[key_seq + key_ann] = tmp
encoding_keys = list(combined_encoding.keys())
encoding_vectors = np.array(list(combined_encoding.values()))
encoding_vectors
#print(len(encoding_vectors))


# Loading in X and y

seq_encoding_keys = list(encoding_seq.keys())
seq_encoding_vectors = np.array(list(encoding_seq.values()))
#annotation_encoding_keys = list(encoding_annotation.keys())
#annotation_encoding_vectors = np.array(list(encoding_annotation.values()))

# Loading in Gene Sequence in a Wrapper

gene_data = Gene_Wrapper.seq_data_loader(False, dataset, lower_bound, upper_bound,
                                            permute=randomization_test) # use_annotations, dataset, lower_bound, upper_bound, permute=randomization_test


encoding_keys = seq_encoding_keys
encoding_vectors = seq_encoding_vectors
X = pad_sequences([[encoding_keys.index(c) for c in gene.seq] for gene in gene_data],
                    maxlen=max_len,
                    dtype=np.int8, value=encoding_keys.index('UNK'))  # , truncating='post')
y = np.array([label_dist(gene.dist) for gene in gene_data])
"""

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        

In [7]:




# Splitting for 5fold

folds_total = 5

kf = KFold(n_splits=10, shuffle=True, random_state=1234)
folds = kf.split(X, y)

# folds now contains a list of lists. Each sublist contains all the indices for the pandas data entries to be used in the respective fold

### **2** |  Importing models...

We use as baseline model the [RNATracker](https://github.com/HarveyYan/RNATracker/blob/master/Models/cnn_bilstm_attention.py) model and a CNN model.


In [8]:
# Import NN
from RNAtracker import RNATracker

# TODO: do a normal CNN


# Set paths for model output
try:
    os.makedirs('~/Downloads/model_outputs')
except Exception as e:
    print(str(e))

model_output_folder = '~/Downloads/model_outputs'



[Errno 17] File exists: '~/Downloads/model_outputs'


In [9]:

# TODO: with understand what we have to predict, we can allocate X and y
# Also: the kwargsvalues are hyperparameters of which we will select default values from the RNAtracker repo
epochs = 10

for i, (train_indices, test_indices) in enumerate(folds):
    print('Evaluating KFolds {}/{}'.format(str(i + 1), str(folds_total)))
    model = RNATracker(max_len, nb_classes, model_output_folder, kfold_index=i)
    #model.build_model(nb_filters=kwargs['nb_filters'], filters_length=kwargs['filters_length'],
    #                          pooling_size=kwargs['pooling_size'], lstm_units=kwargs['lstm_units'],
    #                          embedding_vec=encoding_vectors)
    
    model.build_model_advanced_masking(nb_filters=nb_filters,
                                                   filters_length=filters_length,
                                                   pooling_size=pooling_size,
                                                   lstm_units=lstm_units,
                                                   embedding_vec=encoding_vectors)
    

    model.train(X[train_indices], y[train_indices], batch_size,epochs)

    model.evaluate(X[test_indices], y[test_indices], dataset)

    K.clear_session()







Evaluating KFolds 1/5
Advanced Masking


TypeError: ('Keyword argument not understood:', 'pool_length')